In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()

In [2]:
from pyspark.sql.types import *

In [13]:
from datetime import datetime
from pyspark.sql import Row

In [14]:
def toTimeSafe(inval):
  try:
    return datetime.strptime(inval, "%d-%b-%y %I.%M.%S %p")
  except ValueError:
    return None

In [27]:
def stringToPost(row):
  r = row.split(",")
  return Row(
    r[1].lstrip('\"').rstrip('\"'),r[2].lstrip('\"').rstrip('\"'),
    toTimeSafe(r[4].lstrip('\"').rstrip('\"')),
    toTimeSafe(r[5].lstrip('\"').rstrip('\"'))
    )

In [16]:
dataset = sc.textFile('Irving2200_1month.csv')
dataset = dataset.flatMap(lambda x: x.split('\n'))
#dataset = dataset.map(lambda x: x.split(','))

In [17]:
#header = dataset.first()
#header = [x.lstrip('\"').rstrip('\"') for x in header]
#header

In [18]:
#data = dataset.map(lambda row: [x.lstrip('\"').rstrip('\"') for x in row])
#data = data.filter(lambda x: x != header)

In [19]:
#data.take(1)

In [20]:
dataset_1st = dataset.first()

In [25]:
dataset = dataset.filter(lambda x: x!= dataset_1st)
dataset.take(2)

['"1","490-22370","IRVING ST 2200","CREDIT CARD","01-OCT-18 01.06.55 PM","01-OCT-18 02.06.55 PM","NS",2.25',
 '"2","490-22370","IRVING ST 2200","CASH","01-OCT-18 03.24.30 PM","01-OCT-18 03.31.10 PM","NS",0.25']

In [29]:
revenue = dataset.map(lambda x: stringToPost(x))

In [32]:
schema = StructType([ StructField("meter_id", StringType(), False),
                      StructField("block", StringType(), False),
                      StructField("starttime", TimestampType(), False),
                      StructField("endtime", TimestampType(), False)
                    ])

In [33]:
revenue_df = ss.createDataFrame(revenue, schema)
revenue_df.show()

+---------+--------------+-------------------+-------------------+
| meter_id|         block|          starttime|            endtime|
+---------+--------------+-------------------+-------------------+
|490-22370|IRVING ST 2200|2018-10-01 13:06:55|2018-10-01 14:06:55|
|490-22370|IRVING ST 2200|2018-10-01 15:24:30|2018-10-01 15:31:10|
|490-22370|IRVING ST 2200|2018-10-01 15:37:14|2018-10-01 15:50:34|
|490-22370|IRVING ST 2200|2018-10-01 15:59:00|2018-10-01 16:59:00|
|490-22370|IRVING ST 2200|2018-10-01 16:59:00|2018-10-01 17:29:00|
|490-22370|IRVING ST 2200|2018-10-01 09:01:14|2018-10-01 09:10:34|
|490-22370|IRVING ST 2200|2018-10-01 09:46:39|2018-10-01 10:06:39|
|490-22370|IRVING ST 2200|2018-10-01 10:08:32|2018-10-01 10:15:12|
|490-22370|IRVING ST 2200|2018-10-01 10:11:57|2018-10-01 10:35:12|
|490-22370|IRVING ST 2200|2018-10-01 10:26:35|2018-10-01 11:01:52|
|490-22370|IRVING ST 2200|2018-10-01 11:23:28|2018-10-01 12:12:48|
|490-22370|IRVING ST 2200|2018-10-01 12:17:32|2018-10-01 12:44